In [48]:
%%bash
ExtractRowsWithColumnFilter.py  GBM_DESeq2_DEG.list.with_ENSP.xls   padj      "<0.01"  GBM_DESeq2_DEG.list.with_ENSP.xls.0.01.xls 
wc -l GBM_DESeq2_DEG.list.with_ENSP.xls.0.01.xls 
ExtractSubnetwork.py  /data2/wangb/databases/string_db/protein.actions.v11.0.txt subnet.txt  20200728_101845.txt.KDA_keydriver.xls  20200728_101847.txt.KDA_keydriver.xls GBM_DESeq2_DEG.list.with_ENSP.xls.0.01.xls
wc -l subnet.txt

2252 GBM_DESeq2_DEG.list.with_ENSP.xls.0.01.xls
####################Help:
if __name__ == "__main__":

    import ShowHelp

    path_ppi = sys.argv[1]

    path_output = sys.argv[2]

    paths_proteins = sys.argv[3:]

    df_ppi = readPpi(path_ppi)

    dict_paths_proteinlists = {basename(p):getProteinNamesFromTable(p) for p in paths_proteins}

    edge_ends = ['item_id_a', 'item_id_b']

    for filename_protein, protein_list in dict_paths_proteinlists.items(): 

        for edge_end in edge_ends:

            indices = df_ppi[df_ppi[edge_end].isin(set(protein_list))].index

            print('{} intersect edge_end {} results in {}'.format(filename_protein, edge_end, indices))

            df_ppi.loc[indices, '{}_{}'.format(filename_protein, edge_end)] = 1



    print(df_ppi.head())

    length_of_new_columns = len(dict_paths_proteinlists) * len(edge_ends)

    print(length_of_new_columns)

    df_ppi.loc[:,'sum'] = df_ppi.iloc[:,-length_of_new_columns:].sum(axis=1)

    df_ppi = df_pp

In [56]:
import pandas as pd 
df_in = pd.read_csv('subnet.txt', sep='\t') 
# df_in_filtered =  df_in[(
#                           (df_in['GBM_DESeq2_DEG.list.with_ENSP.xls.0.01.xls_item_id_a'] > 0) &
#                           (df_in['GBM_DESeq2_DEG.list.with_ENSP.xls.0.01.xls_item_id_b'] > 0)
#                          ) |
#                         (
#                           (df_in['20200728_101845.txt.KDA_keydriver.xls_item_id_a'] > 0) |
#                           (df_in['20200728_101845.txt.KDA_keydriver.xls_item_id_b'] > 0) | 
#                           (df_in['20200728_101847.txt.KDA_keydriver.xls_item_id_a'] > 0) |
#                           (df_in['20200728_101847.txt.KDA_keydriver.xls_item_id_b'] > 0)
#                          )
#                        ]  

columns_for_item_id_a = filter(lambda x:x.endswith('_item_id_a'), df_in.columns)
columns_for_item_id_b = filter(lambda x:x.endswith('_item_id_b'), df_in.columns)

df_in.loc[:,"sum_for_item_id_a"] = df_in.loc[:,columns_for_item_id_a].sum(axis=1)
df_in.loc[:,"sum_for_item_id_b"] = df_in.loc[:,columns_for_item_id_b].sum(axis=1)
df_in.loc[:,"both_end_involved"] = df_in.loc[:,['sum_for_item_id_a', 'sum_for_item_id_b']].product(axis=1)

df_in_filtered =  df_in[df_in['both_end_involved'] > 0]
df_in_filtered.shape

(68064, 17)

In [57]:
# import pandas as pd 
# df_in = pd.read_csv('subnet.txt', sep='\t')
# df_in_filtered = df_in[df_in['sum'] > 1.0]
df_in_filtered.to_csv('subset_involved_in_keydriver_or_deg.txt', sep='\t', index=False)

In [52]:
df_in_filtered.shape

(652288, 14)

In [53]:
# Marks.
df_deg = pd.read_csv('GBM_DESeq2_DEG.list.with_ENSP.xls', sep='\t')
df_kda_up = pd.read_csv('20200728_101845.txt.KDA_keydriver.xls', sep='\t')
df_kda_down = pd.read_csv('20200728_101847.txt.KDA_keydriver.xls', sep='\t')

In [54]:
# Filter significant degs. 
padj_threshold = 0.01
df_deg = df_deg[df_deg.padj < padj_threshold]
df_deg.loc[df_deg.log2FoldChange > 0, 'change'] = 'up' 
df_deg.loc[df_deg.log2FoldChange < 0, 'change'] = 'down'
df_deg.to_csv('GBM_DESeq2_DEG.list.with_ENSP_with_up_down.xls', sep='\t', index=False)

In [66]:
# Extract nodes with up / down / keydriver nodes. 
df_deg_nodes = df_deg.loc[:,['gene_id', 'change']].loc[df_deg['gene_id'].str.startswith('ENSP'),:] 


dfs_for_kda_nodes = []
for direction, df_keydriver in [['up', df_kda_up], ['down', df_kda_down]]:
    df_keydriver_nodes = df_keydriver.loc[:,['keydrivers']]
    df_keydriver_nodes.loc[:,'change'] = '{}_keydriver'.format(direction)
    df_keydriver_nodes.columns = ['gene_id','change']
    dfs_for_kda_nodes.append(df_keydriver_nodes)


df_concated_nodes = pd.concat([df_deg_nodes] + dfs_for_kda_nodes, axis=0)
df_concated_nodes.drop_duplicates(subset=['gene_id'], inplace=True, keep='last')
print(df_concated_nodes.shape)
df_concated_nodes.to_excel('nodes_types.xlsx',index=False)

(1199, 2)


In [27]:
%%bash
ExtractSubnetwork.py  /data2/wangb/databases/string_db/protein.actions.v11.0.txt subnet.txt  20200728_101845.txt.KDA_keydriver.xls GBM_DESeq2_DEG.list.with_ENSP_with_up_down.xls

####################Help:
if __name__ == "__main__":

    import ShowHelp

    path_ppi = sys.argv[1]

    path_output = sys.argv[2]

    paths_proteins = sys.argv[3:]

    df_ppi = readPpi(path_ppi)

    dict_paths_proteinlists = {basename(p):getProteinNamesFromTable(p) for p in paths_proteins}

    edge_ends = ['item_id_a', 'item_id_b']

    for filename_protein, protein_list in dict_paths_proteinlists.items(): 

        for edge_end in edge_ends:

            indices = df_ppi[df_ppi[edge_end].isin(set(protein_list))].index

            print('{} intersect edge_end {} results in {}'.format(filename_protein, edge_end, indices))

            df_ppi.loc[indices, '{}_{}'.format(filename_protein, edge_end)] = 1



    print(df_ppi.head())

    length_of_new_columns = len(dict_paths_proteinlists) * len(edge_ends)

    print(length_of_new_columns)

    df_ppi.loc[:,'sum'] = df_ppi.iloc[:,-length_of_new_columns:].sum(axis=1)

    df_ppi = df_ppi[df_ppi['sum'] > 0]

    df_ppi.to_csv(path_out